In [1]:

#code to access all files in a folder given the location of folder.
#and concatenating all features(from all files) into one result dataframe
import pandas as pd
import numpy as np
import os
#rootdir = 'C:/Users/sid/Desktop/test' 
rootdir = 'Data_isha'
file_array = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print(os.path.join(subdir, file)) to print names of all the csv files
        file_array.append(os.path.join(subdir, file))
print(file_array)

#files = [f for f in os.listdir('Data_isha') if os.path.isfile(f)]


merged_df = []
for files in file_array:
    read_file = pd.read_csv(files)
    #dropping columns:
    read_file.drop(['Month','Day', 'Date', 'Year','dewpti','dewpti.1', 'heatindexi',
                    'tempi','windchillm','windchilli','wspdi', 'heatindexm',
                    'wgusti','wdire','pressurei','time_h', 'time_min','precip_ratei',
                    'wdire', 'solarradiation','uv','Station','precip_totali'], axis=1, inplace=True)
    merged_df.append(read_file)
result = pd.concat(merged_df)

print(result.columns.values)


['Data_isha\\IABERDEE25_final.csv', 'Data_isha\\ICORNWAL26_final.csv', 'Data_isha\\IDUNFERM18_final.csv', 'Data_isha\\IEDINBUR99_final.csv', 'Data_isha\\IGLASGOW1_final.csv', 'Data_isha\\IHIGHLAN53_final.csv', 'Data_isha\\IISLEOFS4_final.csv', 'Data_isha\\ILONDON542_final.csv']
['Unnamed: 0' 'tempm' 'dewptm' 'hum' 'wspdm' 'wgustm' 'wdird' 'pressurem'
 'precip_ratem' 'precip_totalm' 'Time']


In [69]:
#step 1 from mail : one regression over all data over all features!-
y = result['tempm']
x1 = result['dewptm']
x2 = result['hum']
x3 = result['wspdm']
x4 = result['wgustm']
x5 = result['wdird']
x6 = result['pressurem']
x7 = result['precip_ratem']
x8 = result['precip_totalm']


reg_result = sm.ols(formula="y ~ x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8", data=result).fit()
print(reg_result.params)

Intercept    17.451640
x1            1.012958
x2           -0.180408
x3            0.002354
x4           -0.000528
x5            0.000187
x6            0.000229
x7            0.013657
x8            0.004173
dtype: float64


In [2]:
#function to access just one file for the time being
import pandas as pd
import numpy as np

def file_read(file):
    for rows in file:
        y = pd.read_csv(file)
        return y
    

In [3]:
df = file_read("Data_isha\\IABERDEE25_final.csv")


In [10]:
#Step 2 : Sliding window of 576 measurements over ALL features

import statsmodels.formula.api as sm
import collections

x1 = 'dewptm'
x2 = 'hum'
x3 = 'wspdm'
x4 = 'wgustm'
x5 = 'wdird'
x6 = 'pressurem'
x7 = 'precip_ratem'
x8 = 'precip_totalm'
y = 'tempm'

sliding_window_values_sens = collections.deque(maxlen=576)
        #iterate over each row of the dataset as individual measurement
for line in df.itertuples():
    
            #extracting the values needed
    values=[getattr(line,x1),getattr(line,x2),getattr(line,x3),getattr(line,x4),getattr(line,x5),getattr(line,x6),getattr(line,x7),getattr(line,x8),getattr(line,y)]  #extracting the values needed
            #appending window
    sliding_window_values_sens.append(values)
    data = list(sliding_window_values_sens)
    window_data=pd.DataFrame(data)
    window_data.columns= [x1,x2,x3,x4,x5,x6,x7,x8,y]
    query = "window_data['tempm'] ~ window_data[x1] + window_data[x2] + window_data[x3] + window_data[x4] + window_data[x5] + window_data[x6] + window_data[x7] + window_data[x8]" #+x3+x4+x5+x6+x7+x8
    
    result = sm.ols(formula=query, data=window_data).fit()
    param_sensor=list(result.params)

print(len(param_sensor))
print(param_sensor)


9
[26.421421991859916, 0.97021166036560824, -0.18057139674413894, -0.0024112643611738371, 0.0040878724040659675, 0.00049566321748589228, -0.0088147778353849637, 0.0, 0.0081287469298900739]
